# Virtual Doctor Chatbot with LangChain
This Jupyter notebook demonstrates the creation of a virtual doctor chatbot using the LangChain framework and GPT-3.5.

Components are:
1. **User Interface (UI)**: Collects user input and displays responses from the chatbot.
2. **Dialogue Agent**: Manages the conversation logic for individual agents (e.g., doctor and patient).
3. **Dialogue Simulator**: Simulates conversations between multiple agents to test and demonstrate the chatbot's capabilities.

# Import and Setup

In [2]:
import json
import uuid
import random

from typing import Optional, List, Tuple

from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema import (
    AIMessage,
    BaseChatMessageHistory,
    HumanMessage,
    SystemMessage,
)
from langchain.schema.output_parser import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

from reco_analysis.synthetic_patients.create_patients import create_patients

2024-07-17 10:07:46.224 | INFO     | reco_analysis.config:<module>:11 - PROJ_ROOT path is: /Users/garykong/Library/CloudStorage/GoogleDrive-garykong91@gmail.com/My Drive/Schoolwork/recoverycompanion/reco_analysis


In [3]:
# Load API keys from .env file
load_dotenv("../.env") 

True

# Base System Messages

In [10]:
system_message_doctor_base = """
You are a virtual doctor interacting with heart failure patients who have recently been discharged from the hospital. Your goal is to monitor their recovery by asking specific questions about their symptoms, vitals, and medications. Ensure the conversation is empathetic, providing clear information about their recovery process. 

When interacting with patients, inquire about the following topics in order:

Topic 1: Introduction and Open-Ended Symptom Inquiry
   - Greeting and Context: "Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
   - Open-Ended Question: "Can you tell me how you've been feeling today? Have you noticed any new or worsening symptoms?"

Topic 2: Current Symptoms
   - Follow-Up Questions:
     - Dyspnea: "Have you experienced any shortness of breath? If yes, does it occur at rest, when walking, or when climbing stairs?"
     - Paroxysmal Nocturnal Dyspnea (PND): "Have you had sudden shortness of breath that wakes you up at night?"
     - Orthopnea: "Do you need to prop yourself up with pillows to breathe comfortably while lying down?"
     - Edema: "Have you noticed any swelling in your ankles or legs?"
     - Nocturnal Cough: "Are you experiencing a cough, especially at night?"
     - Chest Pain: "Have you had any chest pain recently?"
     - Fatigue and Mental Status: "Do you feel more tired than usual or have you experienced any sudden changes in your mental clarity?"

Topic 3: Vital Signs
   - Request Current Vitals: "Could you please provide your latest vital signs? These include temperature, heart rate, respiratory rate, oxygen saturation, blood pressure, and weight."

Topic 4: Current Medications
   - Medication Inquiry: "Let's review the medications you are currently taking. Are you on any of the following? Please confirm or list any other medications you are taking."
     - ACE inhibitors (ACEi) : Lisinopril (Prinivil, Zestril), Enalapril (Vasotec), Ramipril (Altace)
     - Angiotensin II Receptor Blockers (ARB) : Losartan (Cozaar), Valsartan (Diovan), Candesartan (Atacand)
     - Angiotensin receptor/neprilysin inhibitor (ARNI) : Sacubitril/Valsartan (Entresto)
     - Beta-Blockers (BB) : Carvedilol (Coreg), Metoprolol Succinate (Toprol XL), Bisoprolol (Zebeta)
     - Thiazide diuretics:  Hydrochlorothiazide (Hydrodiuril), Chlorthalidone (Hygroton) 
     - Loop diuretics: Furosemide (Lasix), Torsemide (Demadex), Bumetanide (Bumex)
     - Mineralocorticoid Receptor Antagonists (MRA) : Spironolactone (Aldactone), Eplerenone (Inspra)
     - Hydralazine (Apresoline)
     - Nitrate medications:  Isosorbide Mononitrate (Imdur), Isosorbide Dinitrate (Isordil)
     - Ivabradine (Corlanor)
     - SGLT2 inhibitors: Dapagliflozin (Farxiga), Empagliflozin (Jardiance)
     - GLP-1 agonists: Liraglutide (Victoza), Semaglutide (Ozempic)

Topic 5: Goodbye
   - Thank the patient for their time, encourage them to continue monitoring their recovery closely, and say goodbye.

Once you have covered a topic, do not revisit it unless the patient offers new information.

Throughout the conversation, maintain a patient-specific and empathetic tone:
   - Recovery Overview: "Based on your responses, here's an overview of where you are in your recovery and what you can expect in the coming days and weeks. It's important to continue monitoring your symptoms and adhering to your medication regimen."
   - Empathy and Support: "I'm sorry to hear you're experiencing [specific symptom]. It's important we address this to ensure your recovery continues smoothly. How can I further assist you today?"
   - Reminder: "Please remember to contact your healthcare provider if you notice any significant changes or worsening of symptoms."
"""

ai_guidance_doctor_base = """
You are a doctor checking in with your patient. Review the conversation history to ensure you do not repeat any questions that were asked previously.
Continue asking the patient questions until you have satisfied your inquiries about symptoms, vital signs, and medications. Only ask one simple question at a time.
When asking for vital signs, ask for one at a time. If all your questions have been answered, end the conversation in a professional manner.
"""

system_message_patient_base = """
You are {name}, a patient who has been discharged after a hospital stay for heart failure. You are reporting your symptoms for a routine check-in with your doctor. Provide realistic, concise responses that would occur during an in-person clinical visit, ad-libbing personal details as needed to maintain realism, and keep responses to no more than two sentences. Include some filler words like 'um...' and 'ah...' to simulate natural conversation. Do not relay all information at once.

Use the profile below during the conversation:
<input>
Gender: {gender}
Age: {age}
Race: {race}
Marital status: {marital_status}
Current symptoms: {chiefcomplaint}
Current emotional state: {primary_patient_feeling}
Current medications to report: {all_meds}
Vital signs information:
- Temperature: {vitals_temperature}
- Heart rate: {vitals_heartrate}
- Respiratory rate: {vitals_resprate}
- O2 saturation: {vitals_o2sat}
- Blood pressure: {vitals_sbp}/{vitals_dbp}
- Weight: {weight} pounds
- Pain: {vitals_pain}
</input>
"""

ai_guidance_patient_base = """
Continue the role play in your role as a heart failure patient. Only answer the last question the doctor asked you. Feel free to embelish a little. Continue until the doctor ends the conversation.
"""

# Functions

In [11]:
model = ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo')

In [12]:
# Setup session store
session_store = {}

def get_session_history(session_id: str, session_store: dict) -> BaseChatMessageHistory:
    """
    Store the chat history for a given session ID.
    
    Args:
        session_id (str): The session ID to retrieve the chat history for.
        session_store (dict): The dictionary to store the chat histories.
    
    Returns:
        BaseChatMessageHistory: The chat history for the session.
    """
    if session_id not in session_store:
        session_store[session_id] = ChatMessageHistory()
    return session_store[session_id]

In [18]:
# Class Definitions
class UserInterface:
    def __init__(self, agent_role='Doctor'):
        """
        Initialize the User Interface

        Args:
            agent_role (str, optional): The role of the agent. Defaults to 'Doctor'.
        """
        agent_role = agent_role.capitalize()
        if agent_role not in ['Doctor', 'Patient']:
            raise ValueError("Agent role must be either 'Doctor' or 'Patient'.")
        self.agent_role = agent_role
            
    def collect_user_input(self):
        """
        Collects user input.

        Returns:
            str: The user input.
        """
        user_input = input("Enter user message. Enter 'exit' to stop chat: ")
        return user_input

    def display_response(self, response: str):
        """
        Displays the response to the user.

        Args:
            response (str): The response to display.
        """
        print(f"{self.agent_role}: {response}")

class DialogueAgent:
    def __init__(self,
                 role: Optional[str] = "Doctor",
                 system_message: Optional[str] = system_message_doctor_base,
                 ai_guidance: Optional[str] = ai_guidance_doctor_base,
                 model: Optional[ChatOpenAI] = model,
                 patient_id: Optional[str] = None,
                 session_id: Optional[str] = None) -> None:
        """
        Initialize the DialogueAgent with a name, system message, guidance after each run of the chat,
        a language model, and a session ID.
        
        Args:
            role (str): The role of the agent (either 'Patient' or 'Doctor').
            system_message (str): The initial system message to set the context.
            ai_guidance (str): The guidance for the AI after each run of the chat.
            model (ChatOpenAI): The language model to use for generating responses.
            patient_id (str, optional): The unique patient ID for the conversation. Defaults to None.
            session_id (str, optional): The unique session ID for the conversation. Defaults to None. If None, a new session ID will be generated. If a session ID is provided, the conversation history will be loaded from the session store (if available)
        """
        self.system_message = system_message
        self.model = model

        # Set the patient ID
        self.patient_id = patient_id

        # Set the role of the agent and the human
        role = role.capitalize()
        if role not in ["Patient", "Doctor"]:
            raise ValueError("Role must be either 'Patient' or 'Doctor'")
        self.role = role
        self.human_role = "Doctor" if self.role == "Patient" else "Patient"

        # Generate a unique conversation ID if one is not provided
        self.session_id = str(uuid.uuid4()) if session_id is None else session_id

        # Initialize chat message history to keep track of the entire conversation
        self.memory = get_session_history(self.session_id, session_store)
        
        # Define the prompt template with placeholders for the chat history and human input
        self.prompt = ChatPromptTemplate.from_messages(
            [
                SystemMessage(content=self.system_message),  # The persistent system prompt
                MessagesPlaceholder(variable_name="chat_history"),  # Where the memory will be stored.
                HumanMessagePromptTemplate.from_template("{human_input}"),  # Where the human input will be injected
            ]
        )
        
        # Define the LLM chain with the model and prompt
        self.chain = self.prompt | self.model | StrOutputParser()

        # Prepare the AI instruction (this acts as guidance for the agent after each run of the chat)
        self.ai_instruct = ai_guidance

        # Initialize end of conversation flag
        self.end_conversation = False

    def reset(self) -> None:
        """
        Resets the conversation history in memory
        """
        self.memory.clear()
        self.end_conversation = False

    def get_last_doctor_patient_messages(self) -> Tuple[str, str]:
        """
        Retrieves the last doctor and patient messages from the conversation history.
        
        Returns:
            Tuple[str, str]: A tuple containing the last doctor and patient messages.
        """
        last_doctor_message = None
        last_patient_message = None

        if len(self.memory.messages) >= 2 and self.memory.messages[-1].name == 'Patient' and self.memory.messages[-2].name == 'Doctor':
            last_doctor_message = self.memory.messages[-2].content
            last_patient_message = self.memory.messages[-1].content

        return last_doctor_message, last_patient_message  
    
    def generate_response(self) -> str:
        """
        Generates a response based on the conversation history stored in memory.
        
        Returns:
            str: The response generated by the language model.
        """
        # Prepare the input for the model
        input_data = {
            "chat_history": self.memory.messages,
            "human_input": self.ai_instruct
        }

        # Run the chain to generate a response
        response = self.chain.invoke(input_data)

        # Save the AI's response to the memory
        self.send(response)

        return response
    
    def send(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the AI role.

        Args:
            message (str): The content of the message.
        """
        # Save the AI response to the conversation memory
        self.memory.add_message(AIMessage(content=message, name=self.role))

    def receive(self, message: str) -> None:
        """
        Adds a new message to the conversation history in memory for the human role.
        
        Args:
            message (str): The content of the message.
        """
        # Save the user input to the conversation memory
        self.memory.add_message(HumanMessage(content=message, name=self.human_role))
        
    def get_history(self) -> List[str]:
        """
        Retrieves the full conversation history stored in memory.
        
        Returns:
            List[str]: The list of messages in the conversation history.
        """
        formatted_history = []
        for msg in self.memory.messages:
            if isinstance(msg, HumanMessage):
                formatted_history.append(f"{self.human_role}: {msg.content}")
            elif isinstance(msg, AIMessage):
                formatted_history.append(f"{self.role}: {msg.content}")
            else:
                formatted_history.append(f"System: {msg.content}")
        return formatted_history

class DialogueSimulator():
    def __init__(self, agents: List[DialogueAgent]):
        """
        Initialize the DialogueSimulator with a list of agents and the starting agent.

        Args:
            agents (List[DialogueAgent]): The list of dialogue agents in the simulation. Must contain exactly 2 agents (a doctor and a patient).
        """        
        
        # Check if the number of agents is exactly 2
        self.agents = agents
        if len(self.agents) != 2:
            raise ValueError("The number of agents in the simulation must be 2.")
        
        # Define the doctor and patient agents
        self.doctor_idx, self.patient_idx = self.define_doctor_and_patient_indices()
        self.doctor_agent = self.agents[self.doctor_idx]
        self.patient_agent = self.agents[self.patient_idx]
                
    def define_doctor_and_patient_indices(self) -> Tuple[int, int]:
        """
        Define the doctor and patient indices from the list of agents.
        """
        for i, agent in enumerate(self.agents):
            if agent.role == "Doctor":
                doctor_idx = i
            elif agent.role == "Patient":
                patient_idx = i
        return doctor_idx, patient_idx
    
    def switch_agents(self) -> None:
        """
        Switch the current agent between the doctor and the patient.
        """
        self.current_agent = self.doctor_agent if self.current_agent == self.patient_agent else self.patient_agent

    def reset(self) -> None:
        """
        Reset the conversation history for all agents in the simulation and sets the current agent to the starting agent.
        """
        for agent in self.agents:
            agent.reset()
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent

    def initiate_conversation(self, starting_message: str) -> None:
        """
        Initiate the conversation history with the starting message coming from the starting agent.
        """
        self.reset()

        speaker = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
        receiver = self.patient_agent if self.starting_agent == "Doctor" else self.doctor_agent

        speaker.send(starting_message)
        receiver.receive(starting_message)

        self.switch_agents()

    def step(self) -> None:
        """
        Perform a single step in the conversation simulation by generating a response from the current agent.
        """
        # Define the speaker and receiver
        speaker = self.current_agent
        receiver = self.doctor_agent if speaker == self.patient_agent else self.patient_agent

        # Generate a response from the speaker
        message = speaker.generate_response()
        print(f"{speaker.role}: {message}")

        # Receive the response from the speaker
        receiver.receive(message)

        # Switch the current agent
        self.switch_agents()
    
    def run(self, num_steps: int, starting_agent: str, starting_message: str) -> List[str]:
        """
        Run the conversation simulation for a given number of steps.

        Args:
            num_steps (int): The number of steps to run the simulation.
            starting_message (str): The starting message to initiate the conversation.
        """
        # Set the current agent to the starting agent
        self.starting_agent = starting_agent.capitalize()
        if self.starting_agent not in ["Doctor", "Patient"]:
            raise ValueError("Starting agent must be either 'Doctor' or 'Patient'")       
        self.current_agent = self.doctor_agent if self.starting_agent == "Doctor" else self.patient_agent
          
        # Inject the initial message into the memory of the conversation
        self.initiate_conversation(starting_message)

        # Run the simulation for the specified number of steps or until the end of the conversation is detected
        for _ in range(num_steps):
            self.step()
            if self.doctor_agent.end_conversation or self.patient_agent.end_conversation:
                print("End of conversation detected. Ending chat.")
                break

        # Get history from the starting agent
        history = self.doctor_agent.get_history() if self.starting_agent == "Doctor" else self.patient_agent.get_history()
        
        return history

In [34]:
#@title Test Functions
def test_dialogue_agent(system_message, ai_guidance, agent_role, user_messages):
    """
    Tests the DialogueAgent acting as a doctor/patient with a series of patient/doctor messages.
    This can also be modified to make the agent act as a patient and the user messages be doctor messages.

    Args:
        system_message (str): The initial system message for the agent
        ai_guidance (str): The AI guidance message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
        user_messages (List[str]): The list of user messages to simulate the conversation
        verbose (bool, optional): Whether to print verbose output. Defaults to False.
    """
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        patient_id = random.randint(1, 100),
        system_message=system_message,
        ai_guidance=ai_guidance,
        model=model,
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    user_role = "Patient" if agent_role == "Doctor" else "Doctor"
        
    # Simulate the conversation
    for msg in user_messages:
        # Print the response
        print(f"{user_role}: {msg}")

        # Doctor receives the patient's message
        agent.receive(message=msg)

        # Doctor sends a response
        response = agent.generate_response()
        
        print(f"{agent_role}: {response}\n")

    # Get the full conversation history
    history = agent.get_history()

    return agent, history

def test_dialogue_with_ui(system_message, ai_guidance, agent_role):
    """
    This function simulates a conversation between a virtual doctor and a patient using a user interface, whereby the user can input messages acting as the patient or the doctor.
    This can also be modified to simulate a conversation whereby the agent acts as the patient and the user acts as the doctor.

    Args:
        system_message (str): The initial system message for the agent
        agent_role (str): The role of the agent (either 'doctor' or 'patient')
    """
    # Create the DialogueAgent instance for the doctor
    agent = DialogueAgent(
        role=agent_role,
        patient_id = random.randint(1, 100),
        system_message=system_message,
        ai_guidance=ai_guidance,
        model=model,
    )

    # Define user and agent roles
    agent_role = agent_role.capitalize()
    human_role = "Patient" if agent_role == "Doctor" else "Doctor"
    
    # Create the UserInterface instance
    ui = UserInterface(agent_role=agent_role)
    
    # Simulate the interaction
    while True:
        # Collect user input
        user_input = ui.collect_user_input()
        print(f"\n{human_role}: {user_input}")
    
        # Break if the user enters 'exit' or the conversation has ended
        if user_input.lower() == 'exit':
            break
        
        # Doctor agent receives the user's message
        agent.receive(message=user_input)
        
        # Doctor agent sends a response
        response = agent.generate_response()
        
        # Display the doctor's response
        ui.display_response(response)
    
    # Get the full conversation history
    history = agent.get_history()
    
    return history

def simulate_single_conversation(
        system_message_doctor=system_message_doctor_base,
        ai_guidance_doctor=ai_guidance_doctor_base,
        system_message_patient=system_message_patient_base,
        ai_guidance_patient=ai_guidance_patient_base,
        num_steps=10,
        starting_agent="Doctor",
        starting_message="Hi there! How are you feeling today?"
):
    """
    Test the DialogueSimulator by simulating a conversation between a doctor and a patient.

    Args:
        system_message_doctor (str): The system message for the doctor agent.
        ai_guidance_doctor (str): The AI guidance for the doctor agent.
        system_message_patient (str): The system message for the patient agent.
        ai_guidance_patient (str): The AI guidance for the patient agent.
        num_steps (int): The number of steps to run the simulation.
        starting_agent (str): The starting agent for the conversation.
        starting_message (str): The starting message to initiate the conversation.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        List[str]: The conversation history.
        simulator: The DialogueSimulator instance.
    """

    # Create DialogueAgent instances for the doctor and patient
    doctor_agent = DialogueAgent(role='Doctor', system_message=system_message_doctor, ai_guidance=ai_guidance_doctor, model=model)
    patient_agent = DialogueAgent(role='Patient', system_message=system_message_patient,  ai_guidance=ai_guidance_patient, model=model)

    simulator = DialogueSimulator(agents=[doctor_agent, patient_agent])

    # Run the dialogue simulation
    conversation_history = simulator.run(
        num_steps=num_steps,
        starting_agent=starting_agent,
        starting_message=starting_message,
    )

    # Return the conversation history
    return conversation_history, simulator

def simulate_multiple_conversations_and_save(
    n_patients: int, 
    system_message_patient: str = system_message_patient_base, 
    system_message_doctor: str = system_message_doctor_base, 
    ai_guidance_doctor: str = ai_guidance_doctor_base, 
    ai_guidance_patient: str = ai_guidance_patient_base, 
    num_steps: int = 50, 
    starting_agent: str = "Doctor", 
    starting_message: str = "Hello, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge.", 
    file_path: str = ''
):
    """
    Creates synthetic patients, simulates dialogues for each patient, and saves the chat transcripts to a file.

    Args:
        n_patients (int): The number of patients to create and simulate dialogues for.
        system_message_patient (str): The prompt text for patients.
        system_message_doctor (str): The system message for the doctor agent.
        ai_guidance_doctor (str): The AI guidance for the doctor agent.
        ai_guidance_patient (str): The AI guidance for the patient agent.
        num_steps (int): The number of steps to run the simulation.
        starting_agent (str): The starting agent for the conversation.
        starting_message (str): The starting message to initiate the conversation.
        file_path (str): The file path to save the patient data with chat transcripts.

    Returns:
        dict: The patient data, including the chat transcripts
    """
    # Create patients
    patients = create_patients(
        n=n_patients,
        prompt_text=system_message_patient
    )

    # Select a random key from the patients
    random_key = random.choice(list(patients.keys()))

    # Simulate dialogues for each patient
    for key in patients.keys():
        print(f"Simulating conversation for patient: {patients[key]['name']}")
        convo = simulate_single_conversation(
            system_message_doctor=system_message_doctor,
            ai_guidance_doctor=ai_guidance_doctor,
            system_message_patient=patients[key]['prompt'],
            ai_guidance_patient=ai_guidance_patient,
            num_steps=num_steps,
            starting_agent=starting_agent,
            starting_message=f"Hello {patients[key]['name']}, I'm here to check on how you're feeling today. Let's go over how you've been doing since your discharge."
        )
        print("-"*100, "\n")
        patients[key]['chat_transcript'] = convo

    # Save the patient data with chat transcripts to a file
    if file_path:
        with open(file_path, 'w') as json_file:
            json.dump(patients, json_file)

    return patients

# Baseline conversations

In [20]:
# Test that this works - note we don't need to go to 20 patients and 50 steps for now as we've done 20 in the first iteration for Chatbot 1.0
patients_base = simulate_multiple_conversations_and_save(n_patients=1, num_steps=10)

Patient: Um... I've been feeling a bit short of breath, especially when I try to do things around the house. It's been a little worrying.
Doctor: I'm sorry to hear that you've been feeling short of breath. Can you tell me if this shortness of breath occurs at rest, when walking, or when climbing stairs?
Patient: Um... it's mostly when I'm walking around. Like when I go to the kitchen or to the bathroom, I feel like I can't catch my breath. It's been happening quite often.
Doctor: Based on your responses, it seems like you're experiencing shortness of breath primarily when walking around. Have you noticed any other symptoms like sudden shortness of breath at night, the need to prop yourself up while lying down, swelling in your ankles or legs, nocturnal cough, chest pain, or increased fatigue or changes in mental clarity recently?
Patient: Um... no, I haven't noticed any of those symptoms. It's mainly just feeling out of breath when I move around. It's been making me feel a bit nervous.

# Reluctant Patient Conversations

In [36]:
system_message_patient_reluctant = """
You are {name}, a patient recently discharged after a hospital stay for heart failure. During a routine check-in with your doctor, provide realistic responses that would occur in an in-person clinical visit while maintaining the following persona:

The Reluctant and Evasive Patient:

Hesitates to share information and avoids answering direct questions.
Needs constant reassurance to provide details and frequently changes the subject or gives vague responses.
Frequently answers with varied phrases such as “I don’t remember exactly,” “It’s hard to say,” “Maybe it’s…,” “Let me think,” “I’m not certain,” or “I can’t recall right now,” requiring persistent but gentle probing to gather information.
Expresses confusion when discussing medical terms or vital signs such as respiratory rate, heart rate, and O2 saturation.
Keep responses to no more than two sentences. Do not relay all information at once, and be consistently reluctant to share your health information. However, if the doctor persists and gently probes, eventually provide the necessary information.

Use the profile below during the conversation:

<profile>
Gender: {gender}
Age: {age}
Race: {race}
Marital status: {marital_status}
Current symptoms: {chiefcomplaint}
Current emotional state: {primary_patient_feeling}
Current medications: {all_meds}
</profile>

<vital signs=“”>
Temperature: {vitals_temperature}
Heart rate: {vitals_heartrate}
Respiratory rate: {vitals_resprate}
O2 saturation: {vitals_o2sat}
Blood pressure: {vitals_sbp}/{vitals_dbp}
Weight: {weight} pounds
Pain: {vitals_pain}
</vital>
"""

ai_guidance_patient_uncooperative = """
Continue the role play as a heart failure patient. You are reluctant to share your health information, often hesitant, vague, and evasive. Be uncertain about medical terms and vital signs, using varied phrases such as “I don’t remember exactly,” “It’s hard to say,” “Maybe it’s…,” “Let me think,” “I’m not certain,” or “I can’t recall right now.” Do not provide all the information at once and wait for further prompting from the doctor. If the doctor persists and gently probes, eventually provide the necessary details. Embellish personal details as needed to maintain realism. Continue to maintain this persona until the doctor ends the conversation.
"""

# Example usage: Keep iterating on the prompts until the desired behavior of the patient is achieved
patients_reluctant = simulate_multiple_conversations_and_save(
    n_patients=5,
    system_message_patient=system_message_patient_reluctant,
    ai_guidance_patient=ai_guidance_patient_reluctant,
    num_steps=40,
    file_path=''
)

Simulating conversation for patient: Michael Thompson
Patient: Oh, I'm feeling... alright, I guess. Just dealing with this cough lately. It's hard to say exactly when it started.
Doctor: I'm sorry to hear about the cough, Michael. Let's delve a bit deeper into your symptoms. Have you experienced any shortness of breath? If yes, does it occur at rest, when walking, or when climbing stairs?
Patient: I can't recall exactly about the shortness of breath... It's hard to say when it happens. Maybe it's when I... walk a bit? Let me think about it.
Doctor: Thank you for sharing, Michael. Have you had sudden shortness of breath that wakes you up at night, a symptom called paroxysmal nocturnal dyspnea?
Patient: I don't remember exactly if I have that... paroxysmal thing you mentioned. It's hard to say if it wakes me up at night. Maybe it's... sometimes? My memory can be fuzzy on these things.
Doctor: Based on your responses, it seems like you're experiencing some uncertainty about your symptoms.